### Collect keypoints 2

In [1]:
import constants as cs
import globali as gg
import config_loader as cl
import detect_keypoints as dk

In [2]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

### Initialization

In [3]:
def init():
    action_videos = {}
    with open(cs.JSON_PATH, 'r') as file:
        data = json.load(file)    
    for action in data:
        new_list = []
        for instance in action['instances']:
            new_list.append(instance['video_id'])
        action_videos[action['gloss']] = new_list
    return os.listdir(cs.VIDEOS_PATH), action_videos

In [4]:
# initialization
gg.actions, gg.counters = cl.read_yaml_fields()

In [5]:
videoList, action_dict = init()
action_dict_true = {}

In [6]:

for action in action_dict:
    action_dict_true[action] = list()
    for instance in action_dict[action]:
        filename = str(instance) + ".mp4"
        if filename in videoList:
            action_dict_true[action].append(instance)

action_dict_true = dict(sorted(action_dict_true.items(), key=lambda item: len(item[1]), reverse=True))


### Collect Data

In [7]:
def collect_keypoints_from_video(video_path, action, no_frames):
    
    # load values from settings file 
    gg.actions, gg.counters = cl.read_yaml_fields()
    
    # create directories
    if action not in gg.actions:
        gg.counters[action] = 0
    action_folder = os.path.join(cs.KEYPOINTS_FOLDER, action)
    os.makedirs(action_folder, exist_ok=True)
    action_folder2 = os.path.join(cs.VIDEOS_FOLDER, action)
    os.makedirs(action_folder2, exist_ok=True)
    gg.actions.add(action)
    video_folder = os.path.join(action_folder, str(gg.counters[action]))
    os.makedirs(video_folder, exist_ok=True)
    
    # extract frames from video
    frames = dk.extract_frames(video_path, no_frames)   # cs.NUM_FRAME
    
    # Set mediapipe model 
    with dk.mp_holistic.Holistic(min_detection_confidence=0.5,
                                 min_tracking_confidence=0.5) as holistic:
        # for each frame detect keypoints
        for idx, frame in enumerate(frames):
            image, results = dk.mediapipe_detection(frame, holistic)                   # Make detections
            dk.draw_styled_landmarks(image, results)                                   # Draw landmarks 
            keypoints = dk.extract_keypoints(results)                                  # Extract keypoints
            
            # save keypoints in a file
            npy_folder = os.path.join(video_folder, str(idx))
            np.save(npy_folder, keypoints)
            
            # save video in a file
            frame_path = os.path.join(cs.VIDEOS_FOLDER, action,
                                      str(gg.counters[action]))
            os.makedirs(frame_path, exist_ok=True)
            frame_path = os.path.join(cs.VIDEOS_FOLDER, action, 
                                      str(gg.counters[action]), str(idx) + ".jpg")
            cv2.imwrite(frame_path, frame)
            
        # update actions and counters
        gg.counters[action] += 1
        cl.update_yaml_fields(gg.actions, gg.counters)
    

In [8]:
first_elements  = list(action_dict_true.items())[:15]

for key, value in first_elements:
    for video in value:
        video_path = os.path.join(cs.VIDEOS_PATH, video + ".mp4")
        collect_keypoints_from_video(video_path, key, cs.NUM_FRAME)
        

key:  thin
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57919.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57948.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57949.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57934.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57935.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57936.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57937.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\66606.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57939.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57940.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\57941.mp4
D:/Università/Magistrale_Roma_Tre/Deep_Learning/Progetto/PROGETTO/videos\579